6.1 读写文本格式的数据

read_csv 和 read_table 函数

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [3]:
!cat ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [4]:
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


没有标题行的文件

In [7]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

列名可以默认,也可以自己定义

In [8]:
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


将某一列作为 DataFrame 的索引

In [10]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


将多个列做成一个层次化索引

In [11]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [12]:
parsed = pd.read_csv('examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有一些表格可能不是用固定的分隔符去分割字段的

In [13]:
list(open('examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

传递一个正则表达式作为 read_table 的分隔符

In [14]:
result = pd.read_table('examples/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


可以使用 skiprows 跳过文件的第一行、第三行和第四行

In [15]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [16]:
pd.read_csv('examples/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值标记

In [17]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [18]:
result = pd.read_csv('examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values 可以用一个列表或集合的字符串表示缺失值

In [21]:
result = pd.read_csv('examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


字典的各列可以使用不同的 NA 标记值

In [22]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


6.1.1 逐块读取文本文件

设置 pandas 显示的更紧一些

In [23]:
pd.options.display.max_rows = 10

In [25]:
result = pd.read_csv('examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


只显示指定行数

In [26]:
pd.read_csv('examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


要逐块读取文件,可以指定 chunksize

In [28]:
chunker = pd.read_csv('examples/ex6.csv', chunksize=1000)
chunker

迭代处理 ex6.csv,将值计数聚合到'key'列中

In [29]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

<ipython-input-29-f1e3d3ee3945>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

6.1.2 将数据写出到文本格式

In [30]:
data = pd.read_csv('examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


利用 DataFrame 中的 to_csv 方法,将数据写到一个一都好分割的文件中

In [31]:
data.to_csv('examples/out.csv')

In [32]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


也可以设置其他分隔符

In [33]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出结果中会被表示为空字符串

In [34]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


禁用行和列的标签

In [35]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


只写出一部分的列,并按指定的顺序排列

In [36]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series 也有一个 to_csv 方法

In [51]:
dates = pd.date_range(start='1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv', header=False)
!cat examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


6.1.3 处理分隔符格式

In [52]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


对于任何单字符分隔符文件,可以直接使用 Python 内置的 csv模块

In [54]:
import csv
f = open('examples/ex7.csv')
reder = csv.reader(f)
for line in reder:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


读取文件到一个多行的列表中

In [55]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

将这些行分为标题行和数据行

In [56]:
header, values = lines[0], lines[1:]

创建数据列字典

In [57]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

定义 csv 文件的格式

6.1.4 JOSN 数据

In [59]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [60]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

将 Python 对象装换为 JOSN 格式

In [61]:
asjson = json.dumps(result)

将 JSON 对象装换为 DataFrame

In [62]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


pandas.read_json 可以自动将特别格式的 JSON 数据集装换为 Series 或 DataFrame

In [63]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


默认假设 JSON 数组中每个对象是表格中的一行

In [64]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


从 pandas 输出到 JSON

In [66]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


6.1.5 XML 和 HTML:WEB 信息收集

In [67]:
tables = pd.read_html('examples/fdic_failed_bank_list.html')
len(tables)

1

In [68]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


计算按年份计算倒闭的银行数

In [69]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

6.1.6 利用 lxml.objectify 解析 XML

6.2 二进制数据格式

to_pickle 方法将数据以 pickle 格式保存到磁盘上.pickle用于短期存储,难以保证格式永远是稳定的

In [70]:
frame = pd.read_csv('examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [71]:
frame.to_pickle('examples/frame_pickle')

In [72]:
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


6.2.1 使用 HDF5 格式

In [73]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [74]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


HDFStore 支持两张存储模式 fixed 和 table;后者较慢,但是支持使用特殊语法进行查询操作

In [75]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index>=10 and index<=15'])

,a
10,1.007189
11,-1.296221
12,0.274992
13,0.228913
14,1.352917
15,0.886429


In [76]:
store.close

<bound method HDFStore.close of <class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5
>

pandas.read_hdf 函数可以快捷使用以上工具

In [83]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index<5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


6.2.2 读取 Microsoft Excel文件

pandas.read_excel 函数支持读取存储 excel 中的表格型数据

In [87]:
xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [88]:
frame = pd.read_excel('examples/ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


将 pandas 数据写入为 Excel 格式,首先创建一个 ExcelWriter,然后使用 pandas 对象的 to_excel 方法将数据写入其中

In [89]:
writer = pd.ExcelWriter('examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [93]:
frame.to_excel('examples/ex3.xlsx')

6.3 Web APIs 交互

在 GitHub 搜索 30 个最新的 pandas 项目

In [94]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

In [95]:
data = resp.json()
data[0]['title']

'BLD: update min versions #35732'

In [96]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,35733,BLD: update min versions #35732,[],open
1,35732,BLD: minimum versions need update in `pandas/c...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,35731,"BUG: Series has no attribute ""reshape"" after a...","[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,35730,REF: move towards making _apply_blockwise actu...,"[{'id': 49094459, 'node_id': 'MDU6TGFiZWw0OTA5...",open
4,35727,"QST: Any support for df[""x""] = y where y.colum...","[{'id': 1954720290, 'node_id': 'MDU6TGFiZWwxOT...",open
...,...,...,...,...
25,35684,BUG: get UserWarning about FixedFormatter/Fixe...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,35683,BUG: DataFrame.apply returns inconsistent inde...,"[{'id': 697792067, 'node_id': 'MDU6TGFiZWw2OTc...",open
27,35681,BUG/ENH: compression for google cloud storage ...,"[{'id': 76865106, 'node_id': 'MDU6TGFiZWw3Njg2...",open
28,35680,BUG: Different results from Series.replace wit...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


6.4 数据库交互

In [97]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [98]:
con.commit()

插入数据

In [99]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

查询数据

In [100]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

数据导入 DataFrame

In [102]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [104]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


使用 SQLAlchemy 连接 SQLite 数据库,并从之前创建的表读取数据

In [106]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

Exception ignored in: <function ZipFile.__del__ at 0x7fb711684af0>
Traceback (most recent call last):
  File "/Users/advanceqxj/opt/anaconda3/lib/python3.8/zipfile.py", line 1820, in __del__
    self.close()
  File "/Users/advanceqxj/opt/anaconda3/lib/python3.8/zipfile.py", line 1838, in close
    self._write_end_record()
  File "/Users/advanceqxj/opt/anaconda3/lib/python3.8/zipfile.py", line 1942, in _write_end_record
    self.fp.write(endrec)
  File "/Users/advanceqxj/opt/anaconda3/lib/python3.8/zipfile.py", line 780, in write
    n = self.fp.write(data)
AttributeError: 'NoneType' object has no attribute 'write'


,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
